This project is aim to use historical weather data like temperature, humidity, windspeed, visibility and the pressure, and predict whater was rained or not. the target is extracted from `weather` Since the purpose is to predict the rain only.


# 1. Import the libraries 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as  plt
import seaborn as sns
from datetime import timedelta

# 2. Loading the data and data cleaning

In [2]:
df = pd.read_csv('../dataset/weather-sa-2017-2019-clean.csv')
df.shape

(249023, 15)

In [3]:
df.head()

,Unnamed: 0,city,date,time,year,month,day,hour,minute,weather,temp,wind,humidity,barometer,visibility
0,0,Qassim,1 January 2017,00:00,2017,1,1,24,0,Clear,17,11,64%,1018.0,16
1,1,Qassim,1 January 2017,01:00,2017,1,1,1,0,Clear,17,6,64%,1018.0,16
2,2,Qassim,1 January 2017,03:00,2017,1,1,3,0,Clear,15,11,72%,1019.0,16
3,3,Qassim,1 January 2017,04:00,2017,1,1,4,0,Clear,15,11,72%,1019.0,16
4,4,Qassim,1 January 2017,05:00,2017,1,1,5,0,Clear,15,9,72%,1019.0,16


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 249023 entries, 0 to 249022
Data columns (total 15 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Unnamed: 0  249023 non-null  int64  
 1   city        249023 non-null  object 
 2   date        249023 non-null  object 
 3   time        249023 non-null  object 
 4   year        249023 non-null  int64  
 5   month       249023 non-null  int64  
 6   day         249023 non-null  int64  
 7   hour        249023 non-null  int64  
 8   minute      249023 non-null  int64  
 9   weather     249023 non-null  object 
 10  temp        249023 non-null  int64  
 11  wind        249023 non-null  int64  
 12  humidity    249006 non-null  object 
 13  barometer   248951 non-null  float64
 14  visibility  249023 non-null  int64  
dtypes: float64(1), int64(9), object(5)
memory usage: 28.5+ MB


It show that we have 10 Numerical with int and float variable and 5 Categorical variable with object datatype

### 2.1 Lets Drop irrelated and duplicated features

In [5]:
# Unnamed: 0 is irrlevent 
# date and time are duplicated features 
delete_col =['Unnamed: 0','date','time' ]
df.drop(delete_col, axis =1 , inplace =True)

In [6]:
#replace space to NULL if found in the datafram
df = df.replace(" ", np.nan).replace("N/A", np.nan ).replace("NA", np.nan).replace("", np.nan)

### 2.2 Lets convert feature `humidity` to Numerical

In [7]:
df['humidity'] = df['humidity'].str.replace('%',' ').astype('float')


In [8]:
df.isnull().sum()

city           0
year           0
month          0
day            0
hour           0
minute         0
weather        0
temp           0
wind           0
humidity      17
barometer     72
visibility     0
dtype: int64

We have 17 missing values in `humidity`
and 71 missing value in `barometer`

### 2.3 Dealing with missing values

#### I decide to get means of the humidity and barometer per each city to be replaced with missing values.


checking the means for `humidity` and `barometer` for each city.

In [16]:
# Here to display the means per city.
df.groupby(by='city')[['humidity','barometer']].mean().reset_index()


,city,humidity,barometer
0,Assir,47.590154,1023.498147
1,Baha,38.722982,1019.012376
2,EP,45.092703,1008.609077
3,Hail,41.536087,1013.528736
4,Jawf,30.500442,1015.505283
5,Jazan,45.803066,1023.087145
6,Madina,19.971600,1014.141254
7,Mecca,54.158772,1008.713919
8,Najran,45.953645,1023.093818
9,Northern boarder,26.263764,1012.641971


In [17]:
# now filling the missing values with means for each city
df['humidity'] = df['humidity'].fillna(df.groupby('city')['humidity'].transform('mean'))


In [19]:
df['barometer'] = df['barometer'].fillna(df.groupby('city')['barometer'].transform('mean'))


In [20]:
df.isnull().sum()

city          0
year          0
month         0
day           0
hour          0
minute        0
weather       0
temp          0
wind          0
humidity      0
barometer     0
visibility    0
dtype: int64

### 2.4 Extract and create the target 

it is obvious that `weather` in the dataset represents the weather phenomenas, which are in a text format. and multiple phenomena merged together. Since we are predicting the rain only, So we will extract the keywords `rain` and it's synonyms and placed in a new column shows the result whether is rained or not raind on that day.

In [ ]:
lets check that 

In [30]:
df['weather'].unique()

array(['Clear ', 'Sunny ', 'Scattered clouds ', 'Partly sunny ',
       'Passing clouds ', 'Refreshingly cool ', 'Low level haze ',
       'Duststorm ', 'Thunderstorms  Passing clouds ', 'Fog ',
       'Thunderstorms  Partly sunny ', 'Light rain  Partly sunny ',
       'Dense fog ', 'Thunderstorms  Scattered clouds ',
       'Rain  Passing clouds ', 'Extremely hot ', 'Rain  Partly sunny ',
       'Pleasantly warm ', 'Hot ', 'Mild ', 'Overcast ',
       'Rain  Overcast ', 'Smoke ', 'Thunderstorms  Broken clouds ',
       'Heavy rain  Partly sunny ', 'Thunderstorms  Overcast ',
       'Light rain  Overcast ', 'Warm ', 'Thunderstorms  Cloudy ',
       'Drizzle  Overcast ', 'Thunderstorms  Partly cloudy ',
       'Broken clouds ', 'Sandstorm ', 'Partly cloudy ', 'Mostly cloudy ',
       'Rain  Partly cloudy ', 'Rain  Broken clouds ',
       'Rain  Scattered clouds ', 'Haze ', 'Rain  Mostly cloudy ',
       'Hail  Partly sunny ', 'Thundershowers  Passing clouds ',
       'Thunderstorms  Mor

In [36]:
from nltk.corpus import wordnet

synonyms = []

for syn in wordnet.synsets ("rain") :
    for i in syn.lemmas():
        synonyms.append (i.name () )
print (set (synonyms) )


LookupError: 
**********************************************************************
  Resource [93mwordnet[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('wordnet')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/wordnet[0m

  Searched in:
    - '/Users/ALORFI/nltk_data'
    - '/Users/ALORFI/opt/anaconda3/envs/metis/nltk_data'
    - '/Users/ALORFI/opt/anaconda3/envs/metis/share/nltk_data'
    - '/Users/ALORFI/opt/anaconda3/envs/metis/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************
